<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/NNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install torch==1.5.0 torchvision>=0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
!pip3 install numpy
!pip3 install Pillow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transform

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'=0.5.0'   drive   sample_data
/content/drive/.shortcut-targets-by-id/1viBwZM7BIiD8O4LeU-9UQXyV82oTWqh6/ConfidentMachineTranslation/flores
'=0.5.0'			 configs		  language_models
 analysis			 data			  LM_Thresholding.ipynb
 Analysis			 data-bin		  NCD_Analysis.ipynb
 backward_models		 Ensembles		  noisychannel
 BoostedBinaryClassifers.ipynb	 Ensembling		  NoisyChannel.ipynb
 checkpoints			 FLORES.ipynb		  Resources
 ClassificationDataset		 LanguageAnalysis.ipyn

In [0]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.6 GB  | Proc size: 1.8 GB
GPU RAM Free: 11124MB | Used: 317MB | Util   3% | Total 11441MB


In [0]:
from itertools import zip_longest

def datasetReader(featureFile, labelFile):
    files = [featureFile, labelFile]

    X = []
    Y = []

    for lines in zip_longest(*files, fillvalue=''):
        currX, currY = lines[0], float(lines[1].strip("\n"))
        Xarr = []
        features = currX.split()
        for feature in features:
            Xarr.append(float(feature.strip(",").strip("\n")))
        X.append(Xarr)
        Y.append(currY)
    
    return np.array(X), np.array(Y)

def printDatasetClassProp(Y): 
    classes = {}
    total = len(Y)
    for i in Y:
        if i in classes:
            classes[i] += 1
        else:
            classes[i] = 1
    
    for cls in classes:
        print("Proportion in class " + str(cls) + " = " + str(classes[cls]/total))

In [0]:
!ls
!pip3 install numpy
!pip install numpy
import numpy as np
trainFeatures = open("ClassificationDataset/valid/features.txt")
testFeatures = open("ClassificationDataset/test/features.txt")

trainLabels = open("ClassificationDataset/valid/result.txt")
testLabels = open("ClassificationDataset/test/result.txt")

trainX, trainY = datasetReader(trainFeatures, trainLabels)
testX, testY = datasetReader(testFeatures, testLabels)
printDatasetClassProp(trainY)

'=0.5.0'			 configs		  language_models
 analysis			 data			  LM_Thresholding.ipynb
 Analysis			 data-bin		  NCD_Analysis.ipynb
 backward_models		 Ensembles		  noisychannel
 BoostedBinaryClassifers.ipynb	 Ensembling		  NoisyChannel.ipynb
 checkpoints			 FLORES.ipynb		  Resources
 ClassificationDataset		 LanguageAnalysis.ipynb   scripts
Proportion in class 0.0 = 0.8014849550605705
Proportion in class 1.0 = 0.19851504493942945


In [0]:
from torch.utils.data import Dataset, DataLoader
class DatasetLoad(Dataset):
  def __init__(self, X, Y):
    self.x_data = X
    self.y_data = Y
    self.len = len(self.x_data)

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
        
  def __len__(self):
    return self.len

In [0]:
print(trainX.shape, trainY.shape)
#trainConcat = np.concatenate((trainX, trainY.reshape(trainY.shape[0], 1)), axis=1).shape
trainSet = DatasetLoad(X=torch.from_numpy(trainX), Y=torch.from_numpy(trainY))
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=32, shuffle=True)

#testConcat = np.concatenate((testX, testY.reshape(testY.shape[0], 1)), axis=1).shape
testSet = DatasetLoad(X=torch.from_numpy(testX), Y=torch.from_numpy(testY))
testLoader = torch.utils.data.DataLoader(testSet, batch_size=32, shuffle=True)

(2559, 20) (2559,)


In [0]:
class sentenceBLEUClassifierModel(nn.Module):
  def __init__(self):
    super(sentenceBLEUClassifierModel, self).__init__()
    self.linear1 = nn.Linear(20, 160)
    self.linear2 = nn.Linear(160, 80)
    self.linear3 = nn.Linear(80, 2)
  def forward(self, x):
    x = self.linear1(x)
    x = F.relu(x)
    x = self.linear2(x)
    x = F.relu(x)
    x = self.linear3(x)
    x = F.relu(x)
    return F.softmax(x, dim=1)


In [0]:
learning_rate = 0.0005
num_epochs = 250
batch_size = 32

device = torch.device("cuda:0")
model = sentenceBLEUClassifierModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logits.data, 1)[1].view(target.size()).data == target).sum().item()
    accuracy = 100.0 * corrects/batch_size
    return accuracy

In [0]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    for i, (batch, label) in enumerate(trainLoader):
        
        batch = batch.to(device)
        label = label.to(device)

        ## forward + backprop + loss
        logits = model(batch.float())
        loss = criterion(logits, label.long())
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, label.float(), batch_size)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i)) 

Epoch: 0 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 1 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 2 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 3 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 4 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 5 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 6 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 7 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 8 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 9 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 10 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 11 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 12 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 13 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 14 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 15 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 16 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 17 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 18 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 19 | Loss: 0.7019 | Train Accuracy: 81.13
Epoch: 20 | Loss: 0.7019 | Tra

In [0]:
test_acc = 0.0
for i, (batch, label) in enumerate(testLoader):
    batch = batch.to(device)
    label = label.to(device)
    outputs = model(batch.float())
    test_acc += 100*(torch.max(outputs.data, 1)[1].view(label.size()).data == label).sum().item()/batch_size
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 75.14
